<a href="https://www.kaggle.com/code/wcromar/distributed-pytorch-xla-basics-with-pjrt?scriptVersionId=124856098" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Distributed PyTorch/XLA Basics with PJRT (Beta)

Beginning with PyTorch/XLA 2.0, Kaggle supports the new PJRT preview runtime on TPU VMs! For more information about PJRT, see the [PyTorch/XLA GitHub repository](https://github.com/pytorch/xla/blob/master/docs/pjrt.md).

PyTorch/XLA is a package that lets PyTorch run on TPU devices. Kaggle provides a free v3-8 TPU VM. v3-8 TPUs have 8 logical devices: 4 TPU chips, each having 2 cores. This notebook shows how to run simple distributed operations on a TPU using the PJRT runtime. For more information about the Cloud TPU architecture, [see the official documentation](https://cloud.google.com/tpu/docs/system-architecture-tpu-vm).

At the time of writing Kaggle Notebooks on TPU VM are preinstalled with Python 3.8 and PT/XLA 2.0. See below for the exact versions.

In [1]:
!python --version

Python 3.8.16


In [2]:
import torch
torch.__version__

'2.0.0+cu117'

In [3]:
import torch_xla
torch_xla.__version__

'2.0'

Unlike JAX or TF, the convention in PyTorch is to start a separate child process per device to minimize the impact of Python's [Global Interpreter Lock](https://en.wikipedia.org/wiki/Global_interpreter_lock). In eager PyTorch, this means spawning one child process per GPU. For more information, see [PyTorch's distributed training documentation](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html#comparison-between-dataparallel-and-distributeddataparallel).

Due to architectural constraints, it is not possible for more than one process to access a TPU chip simultaneously. Because TPU v3 has two TensorCores cores per TPU chip, that means that each process must drive at least two TPU cores. By default, PyTorch/XLA will spawn 4 processes in total (one per chip), each having two threads (one per TensorCore). This is handled transparently by `xmp.spawn`, which mirrors `mp.spawn`. However, it is important to keep in mind that _all distributed workloads on a TPU v2 or v3 are multithreaded_. The function you pass to `spawn` should be thread-safe.

TPU v4 has a different architecture, where each TPU chip is represented to PyTorch as a single device, so we spawn one process per device as expected.

See the [Cloud TPU documentation](https://cloud.google.com/tpu/docs/system-architecture-tpu-vm) for an in-depth look at TPU architecture.

In [ ]:
!printenv

In [4]:
# Temporary hack: remove some TPU environment variables to support multiprocessing
# These will be set later by xmp.spawn.

import os
os.environ.pop('TPU_PROCESS_ADDRESSES')
os.environ.pop('CLOUD_TPU_TASK_ID')

'0'

In [5]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp

To get the current process/thread's default XLA device, use `xm.xla_device()`. XLA devices are numbered as `xla:i`, where `i` is the index of the device within the current process. Since each process has two devices on a TPU v3, this will be `xla:0` or `xla:1`.

In [7]:
import multiprocessing as mp
lock = mp.Manager().Lock()

def print_device(i, lock):
    device = xm.xla_device()
    with lock:
        print('process', i, device)

To run a function on each TPU device, pass it to `xmp.spawn`. We'll use an `mp.Lock` to prevent `print` statements from overlapping between processes. This make the output clearer, but it is optional.

Note: in interactive notebooks, you must use `start_method='fork'`.

In [8]:
xmp.spawn(print_device, args=(lock,), start_method='fork')

E0406 21:36:48.135562430   24639 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2023-04-06T21:36:48.135542832+00:00"}
E0406 21:36:48.140832291   24683 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2023-04-06T21:36:48.140810769+00:00"}
E0406 21:36:48.140935067   24704 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2023-04-06T21:36:48.140917739+00:00"}
E0406 21:36:48.142011794   24681 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain nam

process 4 xla:0
process 5 xla:1
process 6 xla:0
process 7 xla:1
process 2 xla:0
process 3 xla:1
process 1 xla:1
process 0 xla:0


Note: ignore the errors from `oauth2_credentials.cc`. These will be fixed in a future release.

To run `torch` operations on a TPU, pass the corresponding XLA device in as the `device` parameter. When you pass in an XLA device, the operation is added to a graph, which is executed lazily as needed. To force all devices to evaluate the current graph, call `xm.mark_step()`.

In [9]:
def add_ones(i, lock):
    x = torch.ones((3, 3), device=xm.xla_device())
    y = x + x
    
    # Run graph to compute `y` before printing
    xm.mark_step()
    
    with lock:
        print(i, y)

xmp.spawn(add_ones, args=(lock,), start_method='fork')

E0406 21:37:19.123766014   26177 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2023-04-06T21:37:19.123746276+00:00", grpc_status:2}
E0406 21:37:19.123927544   25887 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2023-04-06T21:37:19.123912017+00:00"}
E0406 21:37:19.126139358   26191 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2023-04-06T21:37:19.126123317+00:00"}
E0406 21:37:19.225654426   26703 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain nam

5 tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], device='xla:1')
3 tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], device='xla:1')
2 tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], device='xla:0')
7 tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], device='xla:1')
6 tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], device='xla:0')
4 tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], device='xla:0')
1 tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], device='xla:1')
0 tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], device='xla:0')


To communicate tensors between TPU devices, use the collective communication operations in `xla_model`, such as `all_gather`.



In [10]:
def gather_ids(i, lock):
    # Create a tensor on each device with the device ID
    t = torch.tensor([i], device=xm.xla_device())
    with lock:
        print(i, t)
    
    # Collect and concatenate the IDs
    ts = xm.all_gather(t)
    xm.mark_step()
    with lock:
        print(i, ts)

xmp.spawn(gather_ids, args=(lock,), start_method='fork')

E0406 21:37:49.090571956   28094 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2023-04-06T21:37:49.090549568+00:00"}
E0406 21:37:49.090749719   27805 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2023-04-06T21:37:49.090730711+00:00"}
E0406 21:37:49.091487416   27801 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2023-04-06T21:37:49.091471223+00:00"}
E0406 21:37:49.095649359   28100 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain nam

0 tensor([0], device='xla:0')
1 tensor([1], device='xla:1')
3 tensor([3], device='xla:1')
2 tensor([2], device='xla:0')
7 tensor([7], device='xla:1')
6 tensor([6], device='xla:0')
5 tensor([5], device='xla:1')
4 tensor([4], device='xla:0')
1 tensor([0, 1, 2, 3, 4, 5, 6, 7], device='xla:1')
0 tensor([0, 1, 2, 3, 4, 5, 6, 7], device='xla:0')
2 tensor([0, 1, 2, 3, 4, 5, 6, 7], device='xla:0')
7 tensor([0, 1, 2, 3, 4, 5, 6, 7], device='xla:1')
3 tensor([0, 1, 2, 3, 4, 5, 6, 7], device='xla:1')
6 tensor([0, 1, 2, 3, 4, 5, 6, 7], device='xla:0')
4 tensor([0, 1, 2, 3, 4, 5, 6, 7], device='xla:0')
5 tensor([0, 1, 2, 3, 4, 5, 6, 7], device='xla:1')


PyTorch/XLA 2.0 also ships with experimental support for the `torch.distributed` using the `pjrt://` `init_method`, including `DistributedDataParallel`.

Because replicas are run multithreaded, the distributed function must be thread-safe. However, the global RNG that `torch` uses for module initialization will give inconsistent results between replicas on TPU v3, since there will be multiple threads concurrently using it. To ensure consistent parameters, we recommend broadcasting model parameters from replica 0 to the other replicas using `pjrt.broadcast_master_param`. In practice, you may also load each replica's parameters from a common checkpoint.

In [11]:
import torch.distributed as dist
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.optim as optim

import torch_xla.experimental.pjrt_backend # Registers `pjrt://` init_method
import torch_xla.experimental.pjrt as pjrt

def toy_model(index, lock):
    device = xm.xla_device()
    dist.init_process_group('xla', init_method='pjrt://')

    # Initialize a basic toy model
    torch.manual_seed(42)
    model = nn.Linear(128, 10).to(device)

    # Optional for TPU v4 and GPU
    pjrt.broadcast_master_param(model)

    # `gradient_as_bucket_view=True` required for XLA
    model = DDP(model, gradient_as_bucket_view=True)

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=.001)

    for i in range(10):
        # Generate random inputs and outputs on the XLA device
        data, target = torch.randn((128, 128), device=device), torch.randn((128, 10), device=device)

        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()

        optimizer.step()
        
        # Run the pending graph
        xm.mark_step()

    with lock:
        # Print mean parameters so we can confirm they're the same across replicas
        print(index, [p.mean() for p in model.parameters()])

xmp.spawn(toy_model, args=(lock,), start_method='fork')

E0406 21:38:19.064130031   30133 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2023-04-06T21:38:19.064111082+00:00", grpc_status:2}
E0406 21:38:19.064175505   30135 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2023-04-06T21:38:19.064159685+00:00", grpc_status:2}
E0406 21:38:19.064493148   30134 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2023-04-06T21:38:19.064475235+00:00"}
E0406 21:38:19.065542428   30136 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain nam

0 [tensor(-0.0005, device='xla:0', grad_fn=<MeanBackward0>), tensor(-0.0019, device='xla:0', grad_fn=<MeanBackward0>)]
6 [tensor(-0.0005, device='xla:0', grad_fn=<MeanBackward0>), tensor(-0.0019, device='xla:0', grad_fn=<MeanBackward0>)]
4 [tensor(-0.0005, device='xla:0', grad_fn=<MeanBackward0>), tensor(-0.0019, device='xla:0', grad_fn=<MeanBackward0>)]
7 [tensor(-0.0005, device='xla:1', grad_fn=<MeanBackward0>), tensor(-0.0019, device='xla:1', grad_fn=<MeanBackward0>)]
5 [tensor(-0.0005, device='xla:1', grad_fn=<MeanBackward0>), tensor(-0.0019, device='xla:1', grad_fn=<MeanBackward0>)]
1 [tensor(-0.0005, device='xla:1', grad_fn=<MeanBackward0>), tensor(-0.0019, device='xla:1', grad_fn=<MeanBackward0>)]
2 [tensor(-0.0005, device='xla:0', grad_fn=<MeanBackward0>), tensor(-0.0019, device='xla:0', grad_fn=<MeanBackward0>)]
3 [tensor(-0.0005, device='xla:1', grad_fn=<MeanBackward0>), tensor(-0.0019, device='xla:1', grad_fn=<MeanBackward0>)]


For a more in-depth look at PyTorch/XLA, see our [API guide](https://github.com/pytorch/xla/blob/master/API_GUIDE.md).